In [1]:
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [2]:
%cd ..

d:\Projekt


d:\Projekt\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [12]:
from src.prep_vid import get_images_tensor
import pyarrow.parquet as pq
from pathlib import Path
from tqdm import tqdm
from src.model import load_model
import torch
import pandas as pd

In [ ]:
data_path = [Path('data/datasets/eval'), Path('data/datasets/test')]
models = ['adaface', 'arcface', 'facenet']

In [ ]:
for path in data_path:
    for model_name in models:
        dataset = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/{model_name}_{dataset}.parquet')



Model: adaface, Dataset: eval: 100%|██████████| 2487/2487 [52:26<00:00,  1.27s/it]  
d:\Projekt\venv\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: eval: 100%|██████████| 2487/2487 [46:18<00:00,  1.12s/it]  
d:\Projekt\venv\Lib\site-packages\facenet_pytorch\models\inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issu

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: facenet, Dataset: test: 100%|██████████| 2476/2476 [39:33<00:00,  1.04it/s]  


In [68]:
df[(df['video_id'] == '94') & (df['person_id'] == 'id07802')]

,video_id,person_id,embedding,frame_id
6342,94,id07802,"[0.062200576, -0.028601408, 0.036356226, 0.006...",0.0
6343,94,id07802,"[0.05768734, -0.035108306, 0.036041643, -0.006...",1.0
6344,94,id07802,"[0.06159673, -0.027372284, 0.046508435, -0.007...",2.0
6345,94,id07802,"[0.061609577, -0.052368198, 0.038780663, -0.00...",3.0


In [71]:
path = 'data/datasets/train/'
data_path = [Path(path+str(i)) for i in range(10)]

In [70]:
for path in data_path:
    for model_name in models:
        dataset = path.parts[-2]
        split = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/{model_name}_{dataset}_{split}.parquet')

Model: adaface, Dataset: train:   1%|▏         | 73/5170 [01:52<2:10:35,  1.54s/it]


RuntimeError: Could not create tensor from given input list